<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/Copy_of_BTC_AAI_CREWAI_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai 'crewai[tools]' -q
!pip install google-generativeai -q
!pip install pykrakenapi -q
!pip install krakenex -q
!pip install langchain-google-genai -q

In [ ]:
!pip install colab-env -q
import colab_env

In [1]:
import json
import random
import time
import requests # Keep requests for general HTTP needs if any, though krakenex will handle Kraken calls
from google.colab import userdata
import google.generativeai as genai
import os # Import the os module to set environment variables
from datetime import datetime
import pytz # For timezone handling

from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool, tool # Import tool from crewai.tools

# FIX: Import LiteLLM directly from the litellm library
import litellm # Import the litellm module itself
from litellm import LiteLLM # Import LiteLLM class from litellm

# Import krakenex and pykrakenapi
import krakenex
from pykrakenapi import KrakenAPI

# Configure the Gemini API with the key from Google Colab userdata
GOOGLE_API_KEY = userdata.get('GEMINI')

# Set the Google API key as an environment variable for LiteLLM to pick up.
# This is a good practice and can act as a fallback.
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# FIX: Set the API key globally for LiteLLm. This is crucial for authentication.
litellm.api_key = GOOGLE_API_KEY

# This genai.configure is for the google.generativeai library directly,
# which might still be useful for other direct API calls, but not for CrewAI's LLM via LiteLLM.
genai.configure(api_key=GOOGLE_API_KEY)


# Configure Kraken API keys from Google Colab userdata
# These keys are typically used for private endpoints (e.g., trading, account balance)
# For public data like ticker, krakenex can be initialized without them,
# but we'll include them here for completeness if private calls were to be added.
KRAKEN_API_KEY = userdata.get('KRAKEN_API_KEY')
KRAKEN_API_SECRET = userdata.get('KRAKEN_API_SECRET')

# Initialize Kraken API client
try:
    api = krakenex.API() # This is the raw krakenex API object, used in the reference
    k = KrakenAPI(api) # This is the pykrakenapi wrapper, which might still be useful
except Exception as e:
    print(f"Warning: Could not initialize KrakenAPI client. Public data fetching might still work. Error: {e}")


# Set model and temperature as attributes after instantiation.
llm = LiteLLM() # Instantiate LiteLLM with no arguments
llm.model = 'gemini/gemini-2.5-flash' # Set the model name as an attribute
llm.temperature = 0.2                     # Set temperature as an attribute


# Helper function for simulating market data (as seen in your reference)
def _simulate_market_data(pair="XBTUSD"):
    """
    Simulates Bitcoin market data when real data cannot be fetched.
    """
    print(f"Simulating market data for {pair}...")
    current_price = round(random.uniform(25000.00, 35000.00), 2)
    past_price_24h = round(random.uniform(current_price * 0.95, current_price * 1.05), 2)
    sentiments = ['bullish', 'bearish', 'neutral', 'volatile']
    sentiment = random.choice(sentiments)

    eastern = pytz.timezone('US/Eastern')
    now_eastern = datetime.now(eastern)
    timestamp_eastern = now_eastern.strftime("%Y-%m-%d %H:%M:%S %Z%z")

    return {
        "currency_pair": pair,
        "current_price": current_price,
        "past_price_24h": past_price_24h,
        "timestamp_est": datetime.now().strftime("%Y-%m-%d %H:%M:%S EST"), # Keeping original format
        "timestamp_eastern": timestamp_eastern,
        "source": "Simulated Data",
        "sentiment": sentiment # Add sentiment to simulated data as well
    }


# --- Define Tools for CrewAI Agents ---

@tool
def get_real_bitcoin_data():
    """
    Fetches current Bitcoin price from Kraken's public API or simulates it if real data fails.
    Returns a dictionary with 'price' and 'sentiment'.
    """
    # FIX: Use Kraken's canonical symbol "XXBTZUSD" as per the reference
    kraken_pair = "XXBTZUSD"

    try:
        # Ensure 'api' (krakenex.API client) is available and properly initialized
        if 'api' not in globals() or api is None:
            print("KrakenAPI client (krakenex.API) not initialized. Falling back to simulated data.")
            return _simulate_market_data(kraken_pair)

        ticker_response = api.query_public('Ticker', {'pair': kraken_pair})

        # DEBUG: Uncomment this line to see the raw response if it fails again
        # print(f"DEBUG: Kraken Ticker Raw Response for {kraken_pair}: {json.dumps(ticker_response, indent=2)}")

        # Check for Kraken's 'error' array first. It's empty on success.
        if 'error' in ticker_response and ticker_response['error']:
            error_message = ticker_response['error'][0] if ticker_response['error'] else "Unknown Kraken API error."
            print(f"Kraken Ticker API Error: {error_message}. Falling back to simulated data.")
            return _simulate_market_data(kraken_pair)
        elif 'result' in ticker_response and kraken_pair in ticker_response['result']:
            ticker_info = ticker_response['result'][kraken_pair]
            current_price = float(ticker_info['c'][0]) # Last trade closed price

            # Simulate sentiment as it's not directly from this public endpoint
            sentiments = ['bullish', 'bearish', 'neutral', 'volatile']
            sentiment = random.choice(sentiments)

            data = {
                "currency_pair": kraken_pair,
                "price": current_price, # Renamed to 'price' for consistency with original tool output
                "sentiment": sentiment,
                "source": "Kraken API"
            }
            print(f"Fetched real market data from Kraken: {json.dumps(data, indent=2)}")
            return data
        else:
            # This else block means no 'error' and either no 'result' or 'result' without the pair
            print(f"Unexpected or empty 'result' in Kraken Ticker API for {kraken_pair}. Falling back to simulated data.")
            return _simulate_market_data(kraken_pair)
    except Exception as e:
        print(f"Failed to fetch real data from Kraken API (general exception for {kraken_pair}): {e}. Falling back to simulated data.")
        return _simulate_market_data(kraken_pair)


# --- Define Agents ---
market_analyst = Agent(
    role='Cryptocurrency Market Analyst',
    goal='Provide a comprehensive analysis of the current Bitcoin market and suggest a trading action.',
    backstory="""You are an expert financial analyst specializing in cryptocurrency markets.
    You have a deep understanding of market dynamics, sentiment, and technical indicators.
    Your goal is to provide clear, actionable insights based on real-time data.""",
    verbose=False,
    allow_delegation=False,
    # Pass the llm instance directly
    llm=llm,
    tools=[get_real_bitcoin_data] # The analyst can use this tool to get data
)

trading_strategist = Agent(
    role='Bitcoin Trading Strategist',
    goal='Based on the market analysis, formulate a simulated Bitcoin trading decision (BUY, SELL, HOLD, or RESEARCH).',
    backstory="""You are a seasoned trading strategist with a focus on Bitcoin.
    You take market analysis and translate it into practical, simulated trading decisions.
    You are cautious and prioritize clear signals before recommending action.""",
    verbose=False,
    allow_delegation=False,
    # Pass the llm instance directly
    llm=llm
)

# --- Define Tasks ---
analyze_market_task = Task(
    description=(
        "1. Use the 'get_real_bitcoin_data' tool to fetch the current Bitcoin price and a simulated market sentiment.\n"
        "2. Based on the fetched data and the user's query: '{user_query}', provide a detailed market analysis.\n"
        "3. Suggest a single trading action: 'BUY', 'SELL', 'HOLD', or 'RESEARCH'.\n"
        "4. Your final answer MUST be a JSON string with 'analysis' and 'suggested_action' keys.\n"
        "Example: {{ \"analysis\": \"Bitcoin is showing strong bullish signs...\", \"suggested_action\": \"BUY\" }}"
    ),
    expected_output="A JSON string containing 'analysis' (string) and 'suggested_action' (string: BUY|SELL|HOLD|RESEARCH).",
    agent=market_analyst,
    # output_json=True, # Removed as it expects a Pydantic BaseModel
)

make_trading_decision_task = Task(
    description=(
        "Based on the 'market_analysis_result' (which is a JSON string from the Market Analyst), "
        "interpret the 'suggested_action' and formulate a clear, simulated trading decision.\n"
        "If the suggested action is 'BUY', state that the agent decides to initiate a simulated BUY order at the given price.\n"
        "If 'SELL', state that the agent decides to initiate a simulated SELL order.\n"
        "If 'HOLD', state that the agent decides to HOLD current assets.\n"
        "If 'RESEARCH', state that the agent decides to conduct further research.\n"
        "Your final answer MUST be a concise string describing the agent's simulated decision."
    ),
    expected_output="A concise string describing the agent's simulated trading decision.",
    context=[analyze_market_task], # This task depends on the output of the analysis task
    agent=trading_strategist
)

# --- Main function to run the Crew ---
def main():
    print("--- Bitcoin, Agentic AI (CrewAI), and LLM Synergy Demo (Python) ---")
    print("This script fetches REAL Bitcoin market data from Kraken, uses a CrewAI team for analysis,")
    print("and an Agentic AI makes a simulated trading decision.")
    print("-" * 60)

    # Removed interactive input and set a default query
    user_query = "What's your analysis of the current Bitcoin market and should I buy or sell?"
    print(f"Using default query: '{user_query}'")

    # Create the Crew
    project_crew = Crew(
        agents=[market_analyst, trading_strategist],
        tasks=[analyze_market_task, make_trading_decision_task],
        process=Process.sequential, # Tasks run in order
        verbose=False,
    )

    print("\nStarting the CrewAI process...")
    # Kick off the crew with the user query as input for the first task
    inputs = {"user_query": user_query}
    result = project_crew.kickoff(inputs=inputs)

    print("\n" + "="*60)
    print("--- CrewAI Process Completed ---")
    print("\nFinal Agentic AI Simulated Decision:")
    print(result)
    print("\n" + "="*60)

    print("\n--- End of Demo ---")
    print("This is a simulated demonstration. No real transactions occur.")

if __name__ == "__main__":
    main()


--- Bitcoin, Agentic AI (CrewAI), and LLM Synergy Demo (Python) ---
This script fetches REAL Bitcoin market data from Kraken, uses a CrewAI team for analysis,
and an Agentic AI makes a simulated trading decision.
------------------------------------------------------------
Using default query: 'What's your analysis of the current Bitcoin market and should I buy or sell?'

Starting the CrewAI process...
Fetched real market data from Kraken: {
  "currency_pair": "XXBTZUSD",
  "price": 116475.0,
  "sentiment": "bearish",
  "source": "Kraken API"
}

--- CrewAI Process Completed ---

Final Agentic AI Simulated Decision:
The agent decides to initiate a simulated SELL order.


--- End of Demo ---
This is a simulated demonstration. No real transactions occur.
